In [168]:
import torch
from torch import nn
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.nn.utils.prune as prune

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import Normalize
from torchmetrics import Accuracy

import torch.optim as optim
from cleverhans.torch.attacks.projected_gradient_descent import (projected_gradient_descent)

import quantus
import captum
from captum.attr import Saliency, IntegratedGradients, NoiseTunnel

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
import copy
import gc

import warnings
warnings.filterwarnings('ignore')

In [170]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [172]:
transformer = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), Normalize(mean=0.3814, std=0.3994)])

train_dataset = datasets.FashionMNIST(root='./datasets', train=True, download = True, transform=transformer)
test_dataset = datasets.FashionMNIST(root='./datasets', train=False, download = True, transform=transformer)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True) # num_workers=4,
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, pin_memory=True)

In [174]:
#for natural and adversarial LeNet Model 
class LeNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_1 = torch.nn.Conv2d(1, 6, 5)
        self.pool_1 = torch.nn.MaxPool2d(2, 2)
        self.relu_1 = torch.nn.ReLU()
        self.conv_2 = torch.nn.Conv2d(6, 16, 5)
        self.pool_2 = torch.nn.MaxPool2d(2, 2)
        self.relu_2 = torch.nn.ReLU()
        self.fc_1 = torch.nn.Linear(256, 120)
        self.relu_3 = torch.nn.ReLU()
        self.fc_2 = torch.nn.Linear(120, 84)
        self.relu_4 = torch.nn.ReLU()
        self.fc_3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool_1(self.relu_1(self.conv_1(x)))
        x = self.pool_2(self.relu_2(self.conv_2(x)))
        x = x.view(x.shape[0], -1)
        x = self.relu_3(self.fc_1(x))
        x = self.relu_4(self.fc_2(x))
        x = self.fc_3(x)
        return x

In [176]:
model = LeNet()
learning_rate = 0.001
epochs = 50 
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [178]:
def evaluate_model(model, data, device):
    model.eval()
    logits = torch.Tensor().to(device)
    targets = torch.LongTensor().to(device)

    with torch.no_grad():
        for x_batch, y_batch in data:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            logits = torch.cat([logits, model(x_batch)])
            targets = torch.cat([targets, y_batch])
    
    return torch.nn.functional.softmax(logits, dim=1), targets

In [13]:
def train_normal(model, epochs):
    model.train()
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            logits = model(x_batch)
            loss = criterion(logits, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluate model!
        if epochs%10==0:
            predictions, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [77]:
model_normal = train_normal(model = model.to(device), epochs = epochs)

Epoch 1/50 - test accuracy: 84.84% and CE loss 0.39
Epoch 2/50 - test accuracy: 86.71% and CE loss 0.66
Epoch 3/50 - test accuracy: 88.12% and CE loss 0.52
Epoch 4/50 - test accuracy: 87.97% and CE loss 0.16
Epoch 5/50 - test accuracy: 88.17% and CE loss 0.37
Epoch 6/50 - test accuracy: 88.92% and CE loss 0.05
Epoch 7/50 - test accuracy: 88.87% and CE loss 0.32
Epoch 8/50 - test accuracy: 87.97% and CE loss 0.34
Epoch 9/50 - test accuracy: 89.63% and CE loss 0.42
Epoch 10/50 - test accuracy: 89.14% and CE loss 0.12
Epoch 11/50 - test accuracy: 88.67% and CE loss 0.05
Epoch 12/50 - test accuracy: 89.57% and CE loss 0.37
Epoch 13/50 - test accuracy: 89.65% and CE loss 0.07
Epoch 14/50 - test accuracy: 90.19% and CE loss 0.33
Epoch 15/50 - test accuracy: 89.70% and CE loss 0.23
Epoch 16/50 - test accuracy: 89.77% and CE loss 0.13
Epoch 17/50 - test accuracy: 89.27% and CE loss 0.13
Epoch 18/50 - test accuracy: 89.28% and CE loss 0.12
Epoch 19/50 - test accuracy: 89.38% and CE loss 0.20
Ep

In [79]:
# Model to GPU and eval mode.
model_normal.to(device)
model_normal.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_normal, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 88.92%


In [81]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "lenet_fmnist.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving the model: {MODEL_SAVE_PATH}")
torch.save(obj=model_normal.state_dict(), f=MODEL_SAVE_PATH)

Saving the model: models\lenet_fmnist.pth


# Train Adversarial Models

In [114]:
model = LeNet()
learning_rate = 1e-4
epochs = 40
eps = [0.01,0.03,0.06,0.0,0.3,0.5]
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [116]:
def evaluate_model(model, data, device):
    model.eval()
    logits = torch.Tensor().to(device)
    targets = torch.LongTensor().to(device)

    with torch.no_grad():
        for x_batch, y_batch in data:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            logits = torch.cat([logits, model(x_batch)])
            targets = torch.cat([targets, y_batch])
    
    return torch.nn.functional.softmax(logits, dim=1), targets

In [118]:
def train_adv(model, epsilon, epochs):
    model.train()
    eps = epsilon
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            x_batch = projected_gradient_descent(model, x_batch, eps, eps/10, 40, np.inf)
            optimizer.zero_grad()
            logits = model(x_batch)
            loss = criterion(logits, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluate model!
        if epochs%10==0:
            predictions, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [120]:
model_adversarial = train_adv(model=model.to(device),
                    epsilon = eps[0], 
                    epochs=epochs)

Epoch 1/40 - test accuracy: 74.91% and CE loss 0.61
Epoch 2/40 - test accuracy: 79.06% and CE loss 0.47
Epoch 3/40 - test accuracy: 81.38% and CE loss 0.84
Epoch 4/40 - test accuracy: 82.89% and CE loss 0.35
Epoch 5/40 - test accuracy: 83.80% and CE loss 0.48
Epoch 6/40 - test accuracy: 84.79% and CE loss 0.23
Epoch 7/40 - test accuracy: 84.83% and CE loss 0.18
Epoch 8/40 - test accuracy: 85.29% and CE loss 0.33
Epoch 9/40 - test accuracy: 86.13% and CE loss 0.33
Epoch 10/40 - test accuracy: 86.52% and CE loss 0.37
Epoch 11/40 - test accuracy: 86.38% and CE loss 0.56
Epoch 12/40 - test accuracy: 87.10% and CE loss 0.11
Epoch 13/40 - test accuracy: 86.97% and CE loss 0.42
Epoch 14/40 - test accuracy: 87.02% and CE loss 0.32
Epoch 15/40 - test accuracy: 87.43% and CE loss 0.17
Epoch 16/40 - test accuracy: 87.34% and CE loss 0.39
Epoch 17/40 - test accuracy: 88.18% and CE loss 0.14
Epoch 18/40 - test accuracy: 88.15% and CE loss 0.36
Epoch 19/40 - test accuracy: 88.11% and CE loss 0.22
Ep

In [126]:
# Model to GPU and eval mode.
model_adversarial.to(device)
model_adversarial.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_adversarial, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 89.50%


In [128]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "lenet_fmnist_adv.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving the model: {MODEL_SAVE_PATH}")
torch.save(obj=model_adversarial.state_dict(), f=MODEL_SAVE_PATH)

Saving the model: models\lenet_fmnist_adv.pth


# L1 Unstructured 

In [32]:
model = LeNet()
learning_rate = 0.001
epochs = 50
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr = learning_rate)

In [34]:
def count_params(model):
    total_parameters = 0
    for layer, param in model.named_parameters():
        total_parameters += torch.count_nonzero(param.data)
    return total_parameters

In [36]:
orig_params = count_params(model)
print(f"Unpruned LeNet-5 model has {orig_params} trainable parameters")

Unpruned LeNet-5 model has 44426 trainable parameters


In [38]:
for layer, param in model.named_parameters():
    print(f"layer.name: {layer} & param.shape = {param.shape}")

layer.name: conv_1.weight & param.shape = torch.Size([6, 1, 5, 5])
layer.name: conv_1.bias & param.shape = torch.Size([6])
layer.name: conv_2.weight & param.shape = torch.Size([16, 6, 5, 5])
layer.name: conv_2.bias & param.shape = torch.Size([16])
layer.name: fc_1.weight & param.shape = torch.Size([120, 256])
layer.name: fc_1.bias & param.shape = torch.Size([120])
layer.name: fc_2.weight & param.shape = torch.Size([84, 120])
layer.name: fc_2.bias & param.shape = torch.Size([84])
layer.name: fc_3.weight & param.shape = torch.Size([10, 84])
layer.name: fc_3.bias & param.shape = torch.Size([10])


In [40]:
for layer_name in model.state_dict().keys():
    print(layer_name, model.state_dict()[layer_name].shape)

conv_1.weight torch.Size([6, 1, 5, 5])
conv_1.bias torch.Size([6])
conv_2.weight torch.Size([16, 6, 5, 5])
conv_2.bias torch.Size([16])
fc_1.weight torch.Size([120, 256])
fc_1.bias torch.Size([120])
fc_2.weight torch.Size([84, 120])
fc_2.bias torch.Size([84])
fc_3.weight torch.Size([10, 84])
fc_3.bias torch.Size([10])


In [42]:
model.state_dict().keys()

odict_keys(['conv_1.weight', 'conv_1.bias', 'conv_2.weight', 'conv_2.bias', 'fc_1.weight', 'fc_1.bias', 'fc_2.weight', 'fc_2.bias', 'fc_3.weight', 'fc_3.bias'])

In [62]:
def compute_sparsity(model):
    conv1_sparsity = torch.sum(model.conv_1.weight == 0) 
    conv2_sparsity = torch.sum(model.conv_2.weight == 0)
    fc1_sparsity = torch.sum(model.fc_1.weight == 0) 
    fc2_sparsity = torch.sum(model.fc_2.weight == 0) 
    fc3_sparsity = torch.sum(model.fc_3.weight == 0) 

    num = conv1_sparsity + conv2_sparsity +fc1_sparsity + fc2_sparsity + fc3_sparsity
    denom = model.conv_1.weight.nelement() + model.conv_2.weight.nelement() + model.fc_1.weight.nelement() + model.fc_2.weight.nelement() + model.fc_3.weight.nelement()

    global_sparsity = num/denom * 100

    return global_sparsity

In [64]:
print(f"LeNet-5 global sparsity = {compute_sparsity(model):.2f}%")

LeNet-5 global sparsity = 0.00%


In [68]:
import torch.nn.utils.prune as prune
for name, module in model.named_modules():
    # prune 20% of weights/connections in for all hidden layaers-
    if isinstance(module, torch.nn.Linear) and name != 'fc_3':
        prune.l1_unstructured(module = module, name = 'weight', amount = 0.2)
    
    # prune 10% of weights/connections for output layer-
    elif isinstance(module, torch.nn.Linear) and name == 'fc_3':
        prune.l1_unstructured(module = module, name = 'weight', amount = 0.1)

In [70]:
print(f"LeNet-5 global sparsity = {compute_sparsity(model):.2f}%")


LeNet-5 global sparsity = 18.66%


In [72]:
new_params = count_params(model)
print(f"Pruned LeNet-5 model has {new_params} trainable parameters")

Pruned LeNet-5 model has 44426 trainable parameters


In [74]:
def evaluate_model(model, data, device):
    model.eval()
    logits = torch.Tensor().to(device)
    targets = torch.LongTensor().to(device)

    with torch.no_grad():
        for x_batch, y_batch in data:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            logits = torch.cat([logits, model(x_batch)])
            targets = torch.cat([targets, y_batch])
    
    return torch.nn.functional.softmax(logits, dim=1), targets

In [78]:
def train_pruned(model, epochs):
    model.train()
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            logits = model(x_batch)
            loss = criterion(logits, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluate model!
        if epochs%10==0:
            predictions, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [80]:
model_l1_unstructured = train_pruned(model = model.to(device), epochs = epochs)

Epoch 1/50 - test accuracy: 84.31% and CE loss 0.44
Epoch 2/50 - test accuracy: 85.19% and CE loss 0.10
Epoch 3/50 - test accuracy: 87.83% and CE loss 0.22
Epoch 4/50 - test accuracy: 88.51% and CE loss 0.21
Epoch 5/50 - test accuracy: 88.55% and CE loss 0.26
Epoch 6/50 - test accuracy: 88.11% and CE loss 0.33
Epoch 7/50 - test accuracy: 88.60% and CE loss 0.16
Epoch 8/50 - test accuracy: 89.35% and CE loss 0.15
Epoch 9/50 - test accuracy: 88.93% and CE loss 0.14
Epoch 10/50 - test accuracy: 89.30% and CE loss 0.29
Epoch 11/50 - test accuracy: 89.28% and CE loss 0.42
Epoch 12/50 - test accuracy: 89.80% and CE loss 0.11
Epoch 13/50 - test accuracy: 89.84% and CE loss 0.32
Epoch 14/50 - test accuracy: 89.71% and CE loss 0.22
Epoch 15/50 - test accuracy: 89.47% and CE loss 0.19
Epoch 16/50 - test accuracy: 89.32% and CE loss 0.22
Epoch 17/50 - test accuracy: 89.91% and CE loss 0.21
Epoch 18/50 - test accuracy: 89.57% and CE loss 0.16
Epoch 19/50 - test accuracy: 89.66% and CE loss 0.10
Ep

In [82]:
# Model to GPU and eval mode.
model_l1_unstructured.to(device)
model_l1_unstructured.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_l1_unstructured, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 88.80%


In [84]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "lenet_fmnist_l1_unstructured.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving the model: {MODEL_SAVE_PATH}")
torch.save(obj=model_l1_unstructured.state_dict(), f=MODEL_SAVE_PATH)

Saving the model: models\lenet_fmnist_l1_unstructured.pth


# Global Pruning

In [180]:
model = LeNet()
learning_rate = 0.001
epochs = 20
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(params=model.parameters(), lr = learning_rate)

In [182]:
parameters_to_prune = (
    (model.conv_1, 'weight'),
    (model.conv_2, 'weight'),
    (model.fc_1, 'weight'),
    (model.fc_2, 'weight'),
    (model.fc_3, 'weight')
)

In [184]:
print(f"LeNet-5 global sparsity = {compute_sparsity(model):.2f}%")

LeNet-5 global sparsity = 0.00%


In [186]:
prune_rates_global = [0.2, 0.3, 0.4, 0.5, 0.6]

In [188]:
def evaluate_model(model, data, device):
    model.eval()
    logits = torch.Tensor().to(device)
    targets = torch.LongTensor().to(device)

    with torch.no_grad():
        for x_batch, y_batch in data:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            logits = torch.cat([logits, model(x_batch)])
            targets = torch.cat([targets, y_batch])
    
    return torch.nn.functional.softmax(logits, dim=1), targets

In [190]:
def train_pruned(model, epochs):
    for iter_prune_round in range(1):
        print(f"\n\nIterative Global pruning round = {iter_prune_round + 1}")
        
        # Prune layer-wise in a structured manner-
        prune.global_unstructured(
            parameters_to_prune,
            pruning_method = prune.L1Unstructured,
            amount = prune_rates_global[iter_prune_round]
            
        )
    
        # Print current global sparsity level-
        print(f"LeNet-5 global sparsity = {compute_sparsity(model):.2f}%")
        
        
        # Fine-training loop-
        print("\nFine-tuning pruned model to recover model's performance\n")
        model.train()
        for epoch in range(epochs):
            for x_batch, y_batch in train_dataloader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                logits = model(x_batch)
                loss = criterion(logits, y_batch)
                loss.backward()
                optimizer.step()
    
            # Evaluate model!
            if epochs%10==0:
                predictions, labels = evaluate_model(model, test_dataloader, device)
                test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
                print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [192]:
model_global = train_pruned(model = model.to(device), epochs = epochs)



Iterative Global pruning round = 1
LeNet-5 global sparsity = 20.00%

Fine-tuning pruned model to recover model's performance

Epoch 1/20 - test accuracy: 83.74% and CE loss 0.51
Epoch 2/20 - test accuracy: 85.77% and CE loss 0.40
Epoch 3/20 - test accuracy: 87.77% and CE loss 0.27
Epoch 4/20 - test accuracy: 88.26% and CE loss 0.20
Epoch 5/20 - test accuracy: 88.08% and CE loss 0.23
Epoch 6/20 - test accuracy: 88.52% and CE loss 0.15
Epoch 7/20 - test accuracy: 89.16% and CE loss 0.24
Epoch 8/20 - test accuracy: 88.82% and CE loss 0.26
Epoch 9/20 - test accuracy: 89.18% and CE loss 0.12
Epoch 10/20 - test accuracy: 89.03% and CE loss 0.33
Epoch 11/20 - test accuracy: 88.93% and CE loss 0.22
Epoch 12/20 - test accuracy: 88.85% and CE loss 0.07
Epoch 13/20 - test accuracy: 89.17% and CE loss 0.16
Epoch 14/20 - test accuracy: 89.19% and CE loss 0.25
Epoch 15/20 - test accuracy: 88.07% and CE loss 0.39
Epoch 16/20 - test accuracy: 89.51% and CE loss 0.13
Epoch 17/20 - test accuracy: 89.1

In [193]:
# Model to GPU and eval mode.
model_global.to(device)
model_global.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_global, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 88.91%


In [194]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "lenet_fmnist_global.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving the model: {MODEL_SAVE_PATH}")
torch.save(obj=model_global.state_dict(), f=MODEL_SAVE_PATH)

Saving the model: models\lenet_fmnist_global.pth


# Layered Structured

In [248]:
model = LeNet()
learning_rate = 0.01
epochs = 20
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr = learning_rate)

In [250]:
def train_pruned(model, epochs):
    for iter_prune_round in range(1):
        print(f"\n\nIterative Global pruning round = {iter_prune_round + 1}")
        
        # Prune layer-wise in a structured manner-
        prune.ln_structured(model.conv_1, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.conv_2, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.fc_1, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.fc_2, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.fc_3, name = "weight", amount = 0, n = 2, dim = 0)
        
        # Print current global sparsity level-
        print(f"LeNet-5 global sparsity = {compute_sparsity(model):.2f}%")
        
        
        # Fine-training loop-
        print("\nFine-tuning pruned model to recover model's performance\n")
        model.train()
        for epoch in range(epochs):
            for x_batch, y_batch in train_dataloader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                logits = model(x_batch)
                loss = criterion(logits, y_batch)
                loss.backward()
                optimizer.step()
    
            # Evaluate model!
            if epochs%10==0:
                predictions, labels = evaluate_model(model, test_dataloader, device)
                test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
                print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [252]:
model_layered_structured = train_pruned(model = model.to(device), epochs = epochs)



Iterative Global pruning round = 1
LeNet-5 global sparsity = 9.86%

Fine-tuning pruned model to recover model's performance

Epoch 1/20 - test accuracy: 81.73% and CE loss 0.48
Epoch 2/20 - test accuracy: 82.79% and CE loss 0.66
Epoch 3/20 - test accuracy: 83.19% and CE loss 1.02
Epoch 4/20 - test accuracy: 84.54% and CE loss 0.50
Epoch 5/20 - test accuracy: 84.17% and CE loss 0.12
Epoch 6/20 - test accuracy: 83.47% and CE loss 0.24
Epoch 7/20 - test accuracy: 84.20% and CE loss 0.28
Epoch 8/20 - test accuracy: 85.24% and CE loss 0.40
Epoch 9/20 - test accuracy: 84.44% and CE loss 0.48
Epoch 10/20 - test accuracy: 78.24% and CE loss 0.55
Epoch 11/20 - test accuracy: 84.52% and CE loss 0.58
Epoch 12/20 - test accuracy: 84.65% and CE loss 0.40
Epoch 13/20 - test accuracy: 82.91% and CE loss 0.36
Epoch 14/20 - test accuracy: 85.28% and CE loss 0.26
Epoch 15/20 - test accuracy: 85.06% and CE loss 0.49
Epoch 16/20 - test accuracy: 84.94% and CE loss 0.53
Epoch 17/20 - test accuracy: 85.62

In [253]:
# Model to GPU and eval mode.
model_layered_structured.to(device)
model_layered_structured.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_layered_structured, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 82.36%


In [254]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "lenet_fmnist_structured.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving the model: {MODEL_SAVE_PATH}")
torch.save(obj=model_layered_structured.state_dict(), f=MODEL_SAVE_PATH)

Saving the model: models\lenet_fmnist_structured.pth
